In [1]:
import urllib.request
from pyunsplash import PyUnsplash
from fastai.vision import *
import random
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import pickle
import hashlib
import hmac
import imghdr
import struct
import time
import urllib
from json import JSONDecodeError
import config
import devices
import logging
import os
import sys
import uuid
import requests
import json
from requests_toolbelt import MultipartEncoder

In [ ]:
class Bot: 
  def __init__(self, username, password, device=None, base_path=''):
    self.username = username
    self.password = password
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Linux; Android 6.0; HTC One M9 Build/MRA58K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36");
    chrome_options.add_argument("--start-maximized");
    chrome_options.add_argument("--disable-infobars")
    self.driver = webdriver.Chrome(options=chrome_options)
    self.driver.set_window_size(750,900)
    # firefox_capabilities = webdriver.DesiredCapabilities.FIREFOX
    # firefox_capabilities['marionette'] = True
    # PROXY = "58.216.202.149:8118"
    # firefox_capabilities['proxy'] = {
    # "proxyType": "MANUAL",
    # "httpProxy": PROXY,
    # "ftpProxy": PROXY,
    # "sslProxy": PROXY
    # }
    # self.driver = webdriver.Firefox(capabilities=firefox_capabilities)
    # self.driver = webdriver.Firefox()

    device = device or devices.DEFAULT_DEVICE
    self.device_settings = devices.DEVICES[device]
    self.user_agent = config.USER_AGENT_BASE.format(**self.device_settings)
    self.base_path = base_path

    self.is_logged_in = False
    self.last_response = None
    self.total_requests = 0

    # Setup logging
    self.logger = logging.getLogger('[instabot_{}]'.format(id(self)))

    if not os.path.exists("./config/"):
      os.makedirs("./config/")  # create base_path if not exists

    log_filename = os.path.join(base_path, 'instabot.log')
    fh = logging.FileHandler(filename=log_filename)
    fh.setLevel(logging.INFO)
    fh.setFormatter(logging.Formatter('%(asctime)s %(message)s'))

    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(logging.Formatter(
      '%(asctime)s - %(levelname)s - %(message)s'))

    self.logger.addHandler(fh)
    self.logger.addHandler(ch)
    self.logger.setLevel(logging.DEBUG)

    self.last_json = None

  def closeBrowser(self):
    self.driver.close()

    def post(self):

        # getting what user wants to post on
        response = [str(x) for x in input("What topic would you like to post on today? \n").split()]
        response = ''.join(response)

        # printing user profile picture ;
        print("Great, firstly here is your user persona")
        my_cmd = 'open /Users/aditya.sharma/Downloads/gan-image-removebg-preview.png'
        os.system(my_cmd)

        # actually fetching the image from unsplash for you
        if response == "":

            print("Oops! Looks like you didn't input any query - picking a random category for you!\n")
            response = random_category()
            print("Your category will be " + response)

        photos = py_un.search(type_='photos', query=response)
        print("\n Now that we have that out of the way, here are a bunch of images related to your query:")
        counter = 0

        for photo in photos.entries:

            # finding photo user likes and wants to post
            answer = input('Do you like this photo? \n')
            self.driver.get(photo.link_download)
            time.sleep(3)
            self.driver.close()
            if answer == 'yes':
                # run photo through model and get hashtag and captioning

                full_path = os.getcwd() + '/' + 'images/' + response + '_' + photo.id + '.jpg'
                image_url = photo.link_download
                urllib.request.urlretrieve(photo.link_download, full_path)

                # TODO: make own model
                response_i = requests.get('https://api.imagga.com/v2/tags?image_url=%s' % image_url,
                                          auth=(api_key_imagga, api_key_imagga_secret))
                tags = []
                print("Great, we really like that picture too! These are some of the tags you could use for captioning \n")
                for i in range(5):
                    tags.append(response_i.json().get("result").get("tags")[i].get("tag").get("en"))
                    print('#' + tags[i] + '\n')

                # TODO: Figure out how to increase classification and generate comments based on classification using AI
                answer = classify(full_path, learn)
                caption = caption_generate(answer, tags)
                self.upload_photo(full_path, caption)
                break
            else:
                continue

